In [17]:
import cv2
import numpy as np
from scipy import ndimage

def removeDuplicates(listofElements):
    uniqueList = []
    for elem in listofElements:
        if elem not in uniqueList:
            uniqueList.append(elem)       
    return uniqueList



def findFomular(img):
    
    margin = 15
    color = img.mean(axis=0).mean(axis=0)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ###### Xu ly theo chieu doc
    ##  threshold
    
    th, threshed = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
    pts = cv2.findNonZero(threshed)
    ret = cv2.minAreaRect(pts)

    (cx,cy), (w,h), ang = ret
    if ang < -50:
        ang = 0
    M = cv2.getRotationMatrix2D((cx,cy), ang, 1.0)
    rotated = cv2.warpAffine(threshed, M, (img.shape[1], img.shape[0]))
    hist = cv2.reduce(rotated,1, cv2.REDUCE_AVG).reshape(-1)

    th = 2
    H,W = img.shape[:2]
    uppers = [y for y in range(H-1) if hist[y]<=th and hist[y+1]>th]
    lowers = [y for y in range(H-1) if hist[y]>th and hist[y+1]<=th]
    
    rotated = cv2.cvtColor(rotated, cv2.COLOR_GRAY2BGR)

    thres = 0
    for i in range(len(uppers)-1):
            thres += (uppers[i+1] - lowers[i])
            
    if len(uppers) > 0:
        thres = thres/len(uppers)

    y_h = []
    y_hs = []
    y_hsss = [] 
    if len(lowers) > 0:
        y_h.append(uppers[0])
        for i in range(len(lowers) -1):
            try:
                dis = uppers[i+1] - lowers[i]
                if dis > thres :
                    y_h.append(lowers[i])
                    y_h.append(uppers[i+1])
            except:
                continue
        y_h.append(lowers[len(lowers)-1])   

    if len(y_h) > 0:
        for i in range(len(y_h) - 1):
            if i%2 == 0:
                y_hs.append([y_h[i], y_h[i+1] - y_h[i]])    
         
        for i in range(len(y_hs) - 1):
            if y_hs[i][0] != y_hs[i+1][0]:
                y_hsss.append(y_hs[i])
        y_hsss.append(y_hs[len(y_hs)-1])

        # remove question: if line'height too small or too big 
        thres_y = 0
        for i in range(len(y_hsss)):
            thres_y += y_hsss[i][1]
        thres_y = thres_y/len(y_hsss)
        
        if len(y_hsss) > 2:
            for box in y_hsss:
                if box[1] < (thres_y * 0.7) or box[1] > (thres_y * 2) :
                    cv2.rectangle(gray, (0, box[0]), (img.shape[1], box[0] + box[1] + margin),color , -1)
                    y_hsss.remove(box)
        cv2.imwrite("result1.png", gray)
        
    ##### Xu ly theo chieu ngang
    
    ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)
    kernel = np.ones((30,30), np.uint8)
    img_dilation = cv2.dilate(thresh, kernel, iterations=1)
    im2,ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])

    x_w = []
    y_m = []
    for i, ctr in enumerate(sorted_ctrs):
        x, y, w, h = cv2.boundingRect(ctr)  
        if (w * h) >  (img.shape[0]*img.shape[1]/50):
            x_w.append([x,w])
            y_m.append(y)
    
    x_wss = []
    thres_x = 0 
    if len(x_w) > 0: 
        
        #remove same box
        for bb in x_w:
            thres_x += bb[1]
        thres_x = thres_x/len(x_w)

        y_moc = []
        x_ws = []
        for i in range(len(x_w)):
            if x_w[i][1] <= thres_x * 1.5 :
                x_ws.append(x_w[i])
                
        #remove question: if wight too long 
        for i in range(len(x_w)):
            if x_w[i][1] > thres_x * 1.5:            
                y_moc.append(y_m[i])

        if len(y_moc)!=0:
            y_hss = []
            for i in range(len(y_hsss)):
                for j in range(len(y_moc)):
                    if abs(y_hsss[i][0] - y_moc[j]) > 30:
                        y_hss.append(y_hsss[i])
            y_hss = removeDuplicates(y_hss)
        else:
            y_hss = y_hsss
        
        x = []
        for n in range(len(x_ws)):
            a = [x_ws[i][1] for i, j in enumerate(x_ws) if abs(j[0] - x_ws[n][0]) <= (thres_x/3)]
            x.append([x_ws[n][0], max(a)])

        for n in range(len(x)):
            a = [x[i][0] for i, j in enumerate(x) if j[1] == x[n][1]]
            x_wss.append([min(a), x[n][1]])    

        x_wss = removeDuplicates(x_wss) 
        
    else:
        y_hss = y_hsss
    
    ### Ket hop [x,w], [y,h] to [x,y,w,h]  
    
    bbox = []
    bb = []
    if len(y_hss) > 0 and len(x_wss) > 0:
        for i in range(len(x_wss)):
            for j in range(len(y_hss)):
                bb = [x_wss[i][0], y_hss[j][0], x_wss[i][1], y_hss[j][1]]
                bbox.append(bb)
    elif len(y_hss) == 0:
        for i in range(len(x_wss)):
            bb = [x_wss[i][0], 0 , x_wss[i][1], img.shape[0]]
            bbox.append(bb)

    elif len(x_wss) == 0:
        for j in range(len(y_hss)):
            bb = [0, y_hss[j][0], img.shape[1], y_hss[j][1]]
            bbox.append(bb)
    
    ### Remove small area
    
    area = 0
    for bb in bbox:
        area += bb[2]*bb[3]
    area = area / len(bbox)
   
    bboxs = []            
    for bb in bbox:
        if (bb[2] * bb[3]) >  (area /3):
            bboxs.append(bb)
    
    bboxss = []            
    for bb in bboxs:
        if bb[2] < (thres_x * 2):
            if (bb[1] - margin) > 0:
                bboxss.append([bb[0], bb[1] - margin, bb[2], bb[3] + 2 * margin])
            else:
                bboxss.append([bb[0], 0, bb[2], bb[3] + margin])
    
    for bb in bboxss:
        cv2.rectangle(img,(bb[0],bb[1]),( bb[0] + bb[2], bb[1] + bb[3]),(90,0,255),2)
    cv2.imwrite("result.png", img)
    return bboxss

In [19]:
img = cv2.imread("/home/topica/Desktop/toan9_test8_noise.jpg")
findFomular(img)

[[403, 598, 1477, 101], [403, 802, 1477, 49]]

In [461]:
import sys
import base64
import requests
import json
import ast
img_b64 ="/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgr/wAARCABnAUQDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD9HsnZUXmFRlqj3vtxvNDEk8mgAefJwKbJcoI9oXmmt16UyZ1H8NAAhYrhqSQYXAFRLIT0JpdxJyWNADcZVkP3WGGXsR70oVVIKqMg5H5YoooAMKFZB91jkr2zRtVpSGUEkYOfQgA/ypQpIyKMEN70DSuTL8ybX6EBT9BS/KF8sfdGcL27f4UyAkpn0OKeSAMmgpKw8cjc3+ec0uPk2+pyaYu4N7cVIQRj3FA2ridIvJH3P7vanxsQpA7jH4UIuOTS4A6CgjlY63VIl2RptUY+UdODmpIxt2sP4TkfWo4+9Sp0/GgOVgCAVjThVyAo6DPPH4k1PDw/H0H49ajRQV561JH096A5WTbVHRR2z+n+FBCnBYZ2nK59ev8AOmZPqaMn1NJuwcrHsWZzIWJYnO49c0KqxndF8pxgY/E/zNIhJHNSw7cZZe+KltspKwy3CxsvlqFCnK47H2/M1k/FP4WeDfjP8L9a+D/jzTbmTQ/EEEcGorYXHlSxoG3B1YY+6a3MIo+7S5ZXyDjgg4PUHt9KQxbWwtNL0mz0S0WQ29jZRWsInfexjiTYpLHPJXrT+N5lH3yAN3sBikQkjJYn60tACxMwAC9B0/z+AqaItFKGhbazHLMP4jyf5mokHGakQ5YH0FCdgJJSzxmJ2LKTuZT0J5/xpPMkDAhz8owPp/8AqokODSU22wFB2lSv8P3famgYUqPuscuvZjT0AOcihwByBSAHlkddjSsVByFPSnRSSKzyrKd8gIdu7AjH9BTUAPJFOoAVnkd2kdtxY7ix65pF+Uqy8Fd2Mds4z+eOaKKAFd2kkWV2LMjbkY9VNPV2cZkck57/AFzUdKMg4BoASY7XwvpzRS5B5bNFAHFyvtO3GfpSFy0O4daDhmywpNyrx/KtDMhZnzgsaQknrTpMbsim0DSuIB2ApajG4fxUuT6mgGrCyZBxSoSRzTN258GpKASuOVgBg0uUdcg0ypLdcrtKigpKwtufkK46tQxPnBR0xmngKvK+uaY3EwJ4wMH8+fyoGSZwc49KlGMgMegqPa2AyoSB1I5+v4D196dbldmRyQcfNQBKcdqWPHem06PvQA6np0/GmU+MZGM0APQ9qkj5GahDqp5NSwsGTigB9GT5uO2aKO+e9TLYCTtUsKllxjvmqwcmWpQxX7rGpAkc/vOOlSP941HCCw5HNSMcnNADk6fjS0idPxp8YHWgATpinUYA6CigB6Et1NLUYJHQ05CT1oAdkjoaehJHNMoyR0NAElFNjJ6GpJMCgBtFFFABSbjvxS/jTkUFeetADsAdBRRRQBwLOzdGNIWY9TTHJHQ0mT6mtCEriuSOlJk+ppMk9TTXJHSgpKw6ikTJHJpaAauIv+tqWohxLipaASsFKGK/dY0lBOO1AyWDLdRTplk+zyfZ94k8tmhfZ8qsI8iT329cd/rxUIcqMhu3Ygj8xkU5bWW+hltbZpUeRDulgjHmoSPlcCQbeDzggZ6YoE3Y8Q+Cknxc8O/tg+KvAPjv456x4v0+38C29+sD6OllYQ3MkzqDGkRG3CKvIwSQefX3uHcAcxbCrsDt5z9K8V8IfsjePfCnxyvf2hz+2B4t1PWdWsU0/VdPv/DOmrYXNmjyMsSoqAx4Ehycda9pl2LO3kKVUHo2cg8ZyD9aATuOpyd6jQk9aeXKjJPTrhScflQMfSoTnGOKiDFhuXJGM9McevNPQvjaqk9OAPU4H68UATOMfeUCliPy4BpgWRMboZVyARhC2QemMdfwp8SSsMqsh4/55kfp1oAeCT/FSbv3n3zinyQzE4Tfn020z7NPncC5HshoauJuw8cSdP1qY4J+UfrUSA7SSx+vk1IobH38c4wYxUWYJ3F3FPusakgJYZ600IRyXf6bDQ0Um7hJGH+6abSQyeM5XNOBI6Gooln2cxkfUVJsm/un8qkCRM4yaX8KZG0oTay/pT0O4bmjbr2U0ALg+3505AQCWGKaYs8C3cfWnKFjTD70PoVNACgZOM0UFl6jPvkYwPx9aazg4CkZJwMHvQA6noSRzUSEnrUidMUAOooooAPMRPvDP0p6MGGQKZjPUCnIPlwDQA6iiigDzvJPU0UUVoQnYKbJ2ockdKbQUncASOhp8Z3DOePWmcd+xwfangSOnl7mHAIAQ5/z79KAbsKAScgH64p652ktng4PHT2qNwixlzCQUONsjsGb1wMVxdz8aJfD/wC0JpXwM8WeDRbW3iLTpZvCfiCO6LR3l1Fw9iw/hk8shw3fP1oC6O3kyDt/MUq79hKD5jnGaa8ax/IGJCnCMf4lBwDSqFZQN+G55P1A/r+hoC6PPP2gP2uf2dv2VNKs9S+OvxDkspLtytpYafo8t7OxHBdkhBYJkEbumQammh+GH7YXwe0zxT8O/ir4x03Q9YkW503XPBV9caVeOqnDRkSAlkzxyce1eX/sP+LH+L/7Svxs+Nd5p8F1Fp2rxeF9CMmCi2se9Zo0zkKjSM25Rwxxmvbdd8H32j/DObwB8EdY0nwVdwWzLo0kejxTWtrIx5kFuCoHrsxtc9RQS22cPZ/sbWtndJNbftefH1pFyR5vxCDhyGyN6+Xzle27PFex267beONXkcLEqBpW3SHaoGWPqSOa8L8BXXxo8N/tQWXwqsv2htf+I/hzTfDc8vxNk8S6BawDRbwKht1hlgiQI7szfuVyUVlOeoHuYcliQRuLZkK9Mn0+lBSViTeVO3Ncb48+NGo+DvFNj4E8J/Arxj421u5snvY7bwzawJHDCrMql5bh0iBJVvlzk4zjkZ67JYehPBJ/u14/+0V8VvhlJrUnwI+J3i34w+CLG90/+0LXxn4BsrhLWaRZCptftMEUgjnzGZNki4I2FSDnADdju/gl8a/BXx68Bf8ACwfBmn6pYxRajLYX+nazAEnsruFtssRIZgwB6FSQegJPFZv7UvxS1H4PfAXxL8QPD929vq0GnmHRZUQMftsxEURwxKyHcQ2CM1h/sTWvj/SP2drfRvH/AIVTTFttavhoskmjCwvNWsDLmG+u4cl/PlHzO7gM7ckDIzn/AB4UfFX9pL4VfACCVWtINQl8X+JMcpFa2Izbo69NrzM4wcD92OKAujm/Dn7HX7cthoOnv4o/4Kv+OBqEmnwSXsEXgjSpI45mjDOisy/NtY45HY1r237JP7adxGFsf+Cq/jpQW++PhzpD4P4qF7j8xXtPjTxbofhnR9W8feKNXFpYadZTajq12Rl4bYDdLJ8/ynGVAHPQ14P8C/DfxB/bEt7b9qD48+Ide0Pwnd3Bf4dfC/RtWmtI5bdG2peanLG4M3mZz5bjbhRQCdy9N+yz+3DHK0Nz/wAFWPEbTocOlz8K9JJXjPoM8c8ZpY/2Zv29I0xB/wAFS7yTPQXHwm0x8/QBv84rrf2gv2uvh58CC2ip4f8AEXi7xXcWguNK8B+CtCmur2aJQw3LtwqRfIwLucYHesz9hD9q7X/20fghefGfxD4GsvDCnX5rOz0axvHuGgijVBtlkY/M+S+QOOnJoG1cyYv2dP8AgoGDvT/gqFESOgb4Qabj/wBCqzH8Av8AgoygxF/wU40lkx0uPg7ZD9EYD9K9xWQyp5g4GccGpYywTahGO/8AnNAkrHhz/Ar/AIKSp8sP/BSTwtIvq/wZgB/8dkFC/Ar/AIKXhf3P/BRfwaB/tfBiH/45XukQVeox+I/wqRQuwBWznpyOf0qZbDPCR8EP+Cl6Ltf/AIKKeBGPo/wYhB/SSkb4Lf8ABTVhkf8ABQX4f/8AhnIv/i6963MvIdgfXk/ifT8akS3adfMdpBzjKxn/AB4OOcdeR6ipA8DHwa/4KaRoN/8AwUH8Abs/w/BuE49+JKlT4Jf8FLTtaH/god4FYfefHwbiOAOp+/wK96hiXa8QDsCAhaKIrjPv0/zxmvFf2mvAvx2+G8+qftV/s0fEfWpNc8P2hvPFHwo17V1n0DxHpsQ/fx20bljZ3Sx/MjxgFj6UAZ8/wX/4KYLIfI/4KL+AlUnEf/FmYjk+n+s/SrFr8Df+CoDjyx/wUQ8Dsxbawh+CiEj8BKP88djj1T4SfFrwj+0J8IvDfxn+GM1y2l+KNM87TVchZ7WRgU8rrt3xzLhlPDCNfUgfLn7XXwa8V/s7/sj3PxHm/bN+K+ufGW71lLbwZr1trz2sOu6pJecWEekljA8KodjbVAGC+7mgD3z4HfDb9szwf4ubU/2gf2qvD/jnQ47F4v7L0f4YrpU0Ux6SGUSE8emK9MUsUxgsu3AZ3LlxnGTlVwc1V0iHWU0fTYfFVqketrpdt/bShFUrdeShlJjXCoRIH4Xtz0Iq6IHyAULBpB5jbd4UEEjCgqTl+PvdxQAIQASSMccg5H0yOlO6dDXzn+z/APHb9orxv/wUQ+Ln7O3irx54b1PwH8PdDsri2lsfDH2S5ju7mIP9nkk3uWcMfvbsf7OK+j0gkkUOkEhUfMDt7f8A1u/pQAJlhyfzpHPOBUiRMwLmJzuJKlV4IGO/49en5io2XEmGYUAKhJHNKCR0NK4C9PSkoAUFsfe/OikooA8/oprkjgGkyfU1oZiydqhkLeZx0qSmyY3ZFBURZAPLMjnhM7B/d4z+NeFftHL+03rvxHtPDmn/ALQOj/Bb4X2+nS3WqfERr7Thf6hc8kW0K3pxEcAHhQfmHOcge57Xcq8PzFH27T0YMCD/ACr4h+NNt4N8D/treJPEX7Z/7HPj34m+HtUsbRfhve6BoL69pVhJt+eOW0WVSkhbc2/GPnAzxVRi5bDlsdP+zP8AHHxJ4U/bB0v4A+Hf+CgR/aB8Ja7pVxNfzX9vaveaBPEoYD7RbDbIGzjHXIPFd9/wUQLeGPhDovxP0yVrbVPCfjfSr60aJirRBpSrpv6nIJzzzux2FeYfCGw8Qal+3r4F+IGg/sMXvwn+H6aPd2fh64j0S3tZb6Q877iOLJt15XHmNu4Nen/8FBLXUfGfhbwh8BtHR59R8X+M7JFgx832OFw9xMR2Cgjr+g5qpQ5dyD6Eubpb9o9R2qv2q3jn2L0XzEDYHtzmqup3i6bpV3f8HyLZ5QT/AHVViT+gqzOlvBttYCDHbRLCrYOCIwEBHr8ozWB8S9QttI+HuvapeSFIrfRbuWU45wIH4A+tQlcD5n/4I0F9S/Z+8ZeILhATqfxDupUbHU+a7E/Unn619IfFvQPiX4y+Gmq+Ffg78ULfwX4iu7VYrDxNcac92LEt9+RYt6gv6MDle2a8N/4JDeFpfD37Eum6vcRtE+u6zd3sat7sMEfTmvpYqJwrKpAC7V+bhQev1oasNK55J+z78Ff2l/hDdaBoHiz9o/wdf+EtG8yTU9C8N+ApNPu9SnZR/pM8yyP5krNySxz8oOAea9nWORMrIoD/ADMVB7MxP8gaqMzlFthIOMkbySUA/iVRyQP4gcg+leDeANN+I/hn9vZvCniT47eKPFunXPw6vdRurTUTFb2NtN5+2MQ2sGI1VVQAF1EhJb5uhKLPoRst8pPB6mrNld6yku60uJ1RABIysyEcELhyyqOUICjkYOe2ajzNG7MygMrZZDxgZx0PNeB/t0/sma7+0T8NfEGoeEfjp410jW7DRHfwt4d0fX2s9LubyJmlLXKrgykgBAu7GACMZNANXPoCeC9E/kzOwlAIP3gVJZVHOTk4ZeQe49K8a/ZriHxG+N/xT/aH25s31OLwl4al7Nb2gLzlPQNLLIG28MUOa5T4B/tlReNf+CcaftC3ECL4g8LaBNpur2JcJIusQD7PEjDcT87uh5wcqOK9a/Zq+Gsvwe/Z48JfD+/V/wC047Bb7WHf7731yz3EzlupbdMRk5PHWgjlZ5N/wVa8SXWk/sUeINPs5nT+3dQ0zSppI5MeXHcTBZOe4wOnevonwvoVn4U8F6H4U02JIrbTNFtbaCGFdqIqRADCjjvnFeJf8FHfhdrfxX/Y88aaN4YtGm1bS7OPV9Pi8k4ke0fzXQEZJdl6L27V6h8BPibonxo+Afgz4t+F51uLLWvD1m52nLQzLGEkRgMkOHUgqeRkHoQaCkrHT3l1dWVhqN9CpDtps6STD/WOhikOxj1I55BP8Rr5h/4I0wCb9iSS7HmbZviBqrKsnPPnKOMjnIFen/tKftX/AAQ/Z08OahpXxG8Q3keq6jolydO0nTfD15eTXDlGRMtDGwXLZHJB4Ptn53/4I6/tL/Cjwx+yxp/wK+IF5rnhjxfL4lurmHTNb8M3qfaY5pNyFXNvjcR/fYD3pN2GfcID7mkjVRtV9zt29MjtXk/x1/bx/ZE/Zg8bQ/Dz47fFiTQ9Wu9MW7htR4fvLjfCzuiyBoYmXlo3GM7vl6cgn1q5ijSZlZCrDORn5l9iOg/A0jvA7BrvTba5dclHubGKVlyBwGbkdPwPOKOZAfO8H/BX7/gmmVDr+006g+vhLUv/AJHrr/gj/wAFCv2K/wBov4g2/wALfgp8bxrmv30Ujw6d/YV7bvKqDJ5mRVH5V6zHFYHAPhqx/wDBLF/8TUySWdtIHt9P063ZVIEsOmxxvgjBAKjIB71LbYEm+RM75zlYypIEe5t2c/u8ZGMcYr5N/wCCivin9sn9n/wlfftjeAP2jorDwb4M17T/AD/h5pfh9Gs77SZZI0uLm/uJAZjODuxFGQCqA/xnP1VdXsenWk+oamjta20TXE6ruLMkSyNhQDknjgd8ivkz4h+LPiN/wVe+BHiX4f8AwF0PxZ4I8AjS7ltf8SeJtLjg1bxNfxo5t9KsLZgG8pyoaW4OCqjA5ZsoD0MXP7UP7Xni3RvjV8IvjHdfCj4SaXqlvd6Xpj6Ox1jxzA3N1LKHJa0sy42xoCQBED0NfRkfkXOqww2oZbeadVBAwpjc7N2OhOOM9vQ18t/8E4P2sdR+OXwR+H/wou/g/wCL7XxL4X0BtN8eajqWmNYWOlNbLtheOS4BE/mgACNMMFX3Jr3v4k/Evwr8Gvhxrvxk8a3sFppfhzTZtRv3kOC5jRyqKHJJcyMihDyScdeKAPnr/gkJdxWH7PvxC8BtYzT6f4S+MOvabplnbBR5sLyGTyY88KzMzhBwu4qGwDmuP+Hfxm/a/T4max+0n8Wf+CWHxZ8SfEBln0/4ewvq2mQaZ4U00ZSKKBJJB/pMu1GnmZcnLqpypFenf8Eu/hv4l+HH7F+l6/4ztJbTXfiDrV94p1SymUrLALyUOkb+pChWAOccccDP0UJ5jG0Uly7q33ldydw27eTnnjjPpx3OQDmta+J9l8PPga3xu+O1ouhx6J4W/tzxjaQMQLKVYw11DHu+Y7ZjtGc5LMRlcY8S/Zu8PfH/APbC+F1v+1L8f/j/AOOfhxB4ojnvvBngXwLqUOmRaJpgVvs9zdSmN2uZ3WNZChGwBh9K6X/gpL8M/iB8Yv2Cvin8NvhTozah4ivvD6zWmnRsfNuxAw/couRksoxjPTg+teRT/tKfGL9ov/gn3L8If2Mvgr4u0LxRpvwwfTPFGreOPDE+nW2gLHafvrS0TcJNQu5mAjiVMKv8RNAF7/gi3pXinVfgB45/aL+J3iu+17xB8RviLdS6jr2pwhLi9tLKRIElbgB2YHdvX5cg8ZGar/t5+M/2qf2OPFPhL9tvXf2p9dvvB9t8SrbRfFXwy0HS4m0y10Cc7EfYE3zXpaN5WcttO5RncW3dL/wTb+K9jov7D/hPw34b/Z68faNoHw/+Hst1rms+J9H/ALPefV4w0kmm20D/AD3U8khKmRNse4Fcb1kJ4j9o/wAO/Hj/AIKu/sieJ9d8IfDLxv8ADTwXoOhSav4O8J+JdOjs9c8ZeIrcCWLzY2O2OwgEZ4HMjr6KpoA9p+H3gX9qL42/GnRP2svHvxz1nwL4Ks9SluPDnwVs7FdmqaW6t9nu9UmRyxmlGZPs7cRr5RABwK91aXdMzggZYHKc4BAPf614p+xP+1R40/ag+GWha5q3wG8Y+F5dH8IWsfirxJ4y02SzhXW1VYVs7WORt9wZFHmM5+QZ6Agk+1OCCV3E7Rs3EfMMADkdOccc0APQ560tR5I6GlyfU0APopEJI5ooA88znrRUZLH7rGnqCOprQzFpsnanU2TtQVEaecZJ46Y/HH86nt9QvreL7PBezRoRyiSkD8B2Hyrxz3qCkJO/igpq5ZnuLm5eSGe6mZZYmS4DyMFZSAAC2ck8dOled+Fvg7qcHx51T49/EHxNp+rXthY/YfBdlpoJTRrNlInnbPWdwzLkfMAqnOeB6ASxABwQGBAPb/GlY72VnLHaxYbmJwf8ffj6U7sjlZI0d07BUdoiW/c+XtIQDgdT1xXzH8Wfjj+1D8WNW+IX7Kfh39iXxXp5vT/Znhzx+1zGNHubOQAT307n5omi5woxnNfSxRDg7enSnSMZohBN80YYMsZ6Bh91v95ex6jtihOwcrMP4QfDHRfhF8JPDXwj8LbLiz8P6RFax3UTZWYqPnlToSGPcgV0qafeycR2cq4OCI8EZ/E1z3xC8FQfEjwjqHgq88Z69oP9pKqtrHhzUDa31v6tHMBlT/nNeSN+wDoEQQQ/tn/tIIGQL83xRfj34g60N3KSse9vpl47Gykspt80eERIzI7k9flGV2r6kc9s15Fp/wCxB4q0z42z/tARftUfFG412bTDps1tcWWmmxksi7OLYQfZhtXcxydxP3eeKsP+x34E1j4Pax8EfFfxp+K2uWGq3kM02q3vxDuE1WAR/djguI41ZVbuqnnvmuIP/BLL9nm2cT2/xY+PBxITJ5fxd1LC9+59OcdeOlIZ9IXnnRTmG4LhkRUYywhWYjI5CnbyR1HHvSRXM9mfNs5SsmcoQ5UbxgKTjgADPbnNZnhXw9ZeD/CemeEdOv8AUbuDTLQQQXOrXrXFxIoJIaWRlVpG56nJ96vkA9O3NAHwvf8A7JnxI8E/8FGtT+DPg2Gdvgv8WNRtfFmp2gtiIluLFi80Ofux73jQj+9X3tqN4uoahNdCTeryExDGNiHO1B9FwKihvLkW7QfaJNj/ACuu8/MOOD7dfpnvSqAVxnp04x2xSbsBJZ3DW04eI4bcpO7kEDqD0yccZ7jtXkngH9lrUvgP8WLnxZ+zt8SP7D8JeJNXe88TfCzW9LFzYtPJgSXGnNHj7DI+1dyjahIJIJJJ9YCqOQKdESoZEJCPgsqkgN9RnFHMgLlvqU1jKz6bdtEhJK7gCzccE7QBwMDA/u57mprnVdSvXZ7jUJZWfG9hIRn8P/r8VSDAMGXjAweM1OHVPvKDUttgJ82QdxwDnB5wfr3pu0B+B+tO+/8Ac49M0vG7OKQCiKIdI1/75FP7cH9KamSOTS0AS2s3luZQCGyMMD07A++ATx3qzPczSGIvdOwhIMSsxITByMDPByTz2HGKpxj3pxJPU0AWrjVNSvIfJld5oyxKQGVv3h3ZOckj6HGfx5ryj4l/su3Px7+I8Wo/H/4nP4i8AaLexXmjfC+y0eWxsZZEO6OfVJC2+/COcrGMrlQSD0r00k5OB1+8c4J/wp0QVW3qOcnn+f5+nGKALclwbu4a4mAMhG1x5aIFXoFARQgVRwAoA5zjJJKUy3BWIKBgD7qjov4d6fQBJDK6FXR2DIwKlWxtPqO4+mcVNPqmqTzK8+pXDsi4UvMzYHoMkkDvVXJHQ09Tke9AE8t/eTXH2uS7lM2ADKZMsR6Enr29zimvLLcyGe4kLyFslyST25yT1OF57bR14xHQCR0NAFuW8ury6F9eXUs0y52zSyFnAPYE5xxxnqefXhgcRoqLwFUKq9dqjoAf68VFGx9aU5oAeGDdKWmxj3p34UAPTp+NFIrADk0UAedoBjNOxxmiitDMKbIKKKBp2G0vG7OKKKCk7jkyRyaWiigYUUUUADBTnKg565p0Sg5znn/aNFFAFDxlpviHxB4O1HRvCXjmTw/qV3p8q6fri2guDp8o+7cLEcK7D+61eJ6D4V+KngH9r3wX8Ivhp+0Z468WuujT33xQt/HWpRz2C23kho57eIqSkryCYqkZ2jaN/BFFFAH0FKpUmXyWiDDIRwARxnHDMP1pKKKTdgJIuUz75qdQQOaKKltsBacg4zRRSAdSqzE4zRRQBPRRRQA9On40tFFAACR0NOQk9aKKAHU9On40UUAOQkPgMamjy3WiigALgHGDQrZ6UUUASISRzS0UUAAJHQ09MkcmiigBQSOhp6v/AHqKKAF60UUUAf/Z"
image_uri = "data:image/jpg;base64," + img_b64
r = requests.post("https://api.mathpix.com/v3/latex",
    data=json.dumps({'url': image_uri}),
    headers={"app_id": "jcarroll", "app_key": "13f1584b2f9edb8220bf619c0b4e3d5a",
            "Content-type": "application/json"})
res = ast.literal_eval(json.dumps(json.loads(r.text), indent=4, sort_keys=True))
res

{'detection_list': ['is_printed'],
 'detection_map': {'contains_chart': 0,
  'contains_diagram': 0.0003,
  'contains_graph': 0.0001,
  'contains_table': 0,
  'is_blank': 0.0004,
  'is_inverted': 0,
  'is_not_math': 0.0004,
  'is_printed': 1},
 'error': '',
 'latex': '\\int _ { - 1 } ^ { \\frac { 1 } { 2 } } \\sqrt [ 3 ] { ( 1 - x ) ^ { 2 } } d x',
 'latex_confidence': 0.673349575622213,
 'latex_confidence_rate': 0.9906467795372009,
 'latex_list': [],
 'position': {'height': 65, 'top_left_x': 64, 'top_left_y': 27, 'width': 235}}